# LAB #2: Numpy

## Introduction
In this laboratory, you will perform some operation with NumPy arrays in such a way to build your first Machine Learning model. 
In particular, you will build a NumPy-based version of the K-Nearest Neighbors algorithm (a.k.a. KNN).

## 0 Preliminary steps
### 0.1 NumPy
Make sure you have the NumPy library installed, its use is strongly recommended for this laboratory.
NumPy is the fundamental package for scientific computing with Python. You can read more about it on
the official documentation.


In [1]:
! pip install numpy


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 0.2 Iris dataset download 
For this lab, you will need two of the datasets you have already met: Iris and MNIST. Please refer to
Laboratory 1 for a complete description of the datasets.
Iris. You can download it from:
https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

In [2]:
# linux users
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data -O iris.csv
# windows users
! pip install wget
import wget
import os
IRIS_DATA_FILE = "iris.csv"
if not os.path.exists(IRIS_DATA_FILE):
    wget.download("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", IRIS_DATA_FILE)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1 Exercises 
Note that exercises marked with a ($\star$) are optional, you should focus on completing the other ones first.

## 1.1 Iris Analysis with Numpy
As you might remember from Lab. 1, the Iris dataset collects the measurements of different Iris flowers,
and each data point is characterized by 4 **features** (sepal length, sepal width, petal length, petal width) and is associated to 1 **label** (i.e. an Iris species - Setosa, Versicolor, or Virginica) which in this case is the last element of the row (last column of the csv file). 

1. Load the Iris dataset. You can use the `csv` library that we saw in the last laboratory or read it with the standard `open(filename, strategy)`. 
In the second case remember to split correctly the different fields, and avoid new line characters. In any case check for empty lines. 
This time remember to store the 4 features in a numpy array `x` of shape (n_sample, 4) and the labels in a different array `y` of shape (n_sample,) converting the 3 different species to a corresponding numerical value. E.g.,
      - Iris-setosa: 0
      - Iris-versicolor: 1
      - Iris-virginica: 2

In order to check you have correctly loaded the data, print the shape of the two arrays: you should find
(150, 4) for `x` and (150,) for `y`.

In [3]:
import csv
import numpy as np

label_mapping = lambda x: 0 if x == "Iris-setosa" else 1 if x == "Iris-versicolor" else 2

data = []
labels = []
with open(IRIS_DATA_FILE) as f:
    for line in csv.reader(f):
        if len(line) == 0: continue
        data.append((list(map(float, line[:-1]))))
        labels.append(label_mapping(line[-1]))

data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)

(150, 4) (150,)


2. Compute again the mean and standard deviation for each class by means of the numpy functions

In [4]:
import numpy as np

means_and_std = {}
for label in np.unique(labels):
    x_class = data[labels == label]
    means_and_std[label] = {
        "mean":x_class.mean(axis=-2),
        "std":x_class.std(axis=-2)
    }

[print(l, d["mean"], d["std"], sep="\n", end="\n\n") for l, d in means_and_std.items()]


0
[5.006 3.418 1.464 0.244]
[0.34894699 0.37719491 0.17176728 0.10613199]

1
[5.936 2.77  4.26  1.326]
[0.51098337 0.31064449 0.46518813 0.19576517]

2
[6.588 2.974 5.552 2.026]
[0.62948868 0.31925538 0.54634787 0.27188968]



[None, None, None]

3. Compute the distances among two samples (e.g., the $36^{th}$ and the $81^{th}$, the $13^{th}$ and the $15^{th}$) 
by means of the `np.linalg.norm(a-b)` function which computes the norm of `a-b`, i.e., the euclidean distance between the feature of the `a` and of the `b` samples. 
  - Can you guess if the two couples of samples belong to the same species?
  - From the mean and standard deviations computed before can you guess which species? 

In [5]:
import numpy as np

samples = (
    (35, 80),
    (12, 14)
)

for (p, q) in samples:
    res = np.linalg.norm(data[p]-data[q])
    print((p,q), res)

(35, 80) 2.908607914449797
(12, 14) 1.4317821063276353


La prima coppia è diversa in quando le features sono distanti tra di loro
La seconda è lo stesso fiore in quanto sono più vicini

4. Find the k nearest neighbors of a sample in the dataset.
    - Define a function `k_nearest_neighbors(x, x_set, k)` that takes as input a sample `x` and a set of sample (i.e., a matrix) `x_set` and returns the indices of the `k` nearest neighbors of `x` in `x_set`.
        - Reuse the `euclidean_distance` function that you defined before to do so. 
        - Remember that the `x_set` is a matrix of shape ($N_{samples}, N_{features}$), so you have to compute the distance between `x` and each row of `x_set`. 
        - In order to find the indices of the `k` nearest neighbors, you can use the `argsort` function that returns the indices that would sort an array
    - Apply the function to the $36^{th}$ sample of the dataset with $k=5$.
    - Print the indices of the $5$ nearest neighbors.
    - Print the labels of the $5$ nearest neighbors. Can you guess the label of the $36^{th}$ sample?

In [6]:
import numpy as np

def k_nearest_neighbors(x, x_set, k):
    distances = np.linalg.norm(x - x_set, axis=-1)
    # print(distances.shape)
    return distances.argsort()[:k]

k_nearest_neighbors(data[35], data, 5)

array([35, 49,  1,  2, 40])

The nearest is itself
Also the others are from the same category

## 1.2 KNN design and implementation
In this exercise, you will implement your own version of the K-Nearest Neighbors (KNN) algorithm, and you will use it to assign an
Iris species (i.e. a label) to flowers whose species is unknown.

The KNN algorithm is straightforward. Suppose that some measurements (e.g., the iris features) and their
relative label (e.g., the iris species) of a set of samples are known in advance. 

<img src="https://mlarchive.com/wp-content/uploads/2022/09/img2.png" width="800">

Then, whenever we want to label a new sample, we look at the K most similar points (a.k.a. neighbors) and assign a label accordingly. 

<img src="https://mlarchive.com/wp-content/uploads/2022/09/img1-1.png" width="800">


The simplest solution is using a majority voting scheme: if the majority of the neighbors votes for a label, we will go for it. 
This approach is naive only at first sight: the local similarity assumed by KNN happens to be roughly true, as you have seen in the previous exercises.
Even though this reasoning does not generalize well, the KNN provides a valid baseline for your tasks.


1. Let’s identify a portion of our data for which we will try to guess the species. Randomly select 20%
of the records and store the first four columns (i.e. the features representing each flower) into a
two-dimensional numpy array of shape ($N_{test}, 4$), you can call it `X_test` and $N_{test}$ is the 20% of the total number of samples.
For the same records, store the test label column (i.e. the one with the species values) into another array, namely `y_test`. 
This is the data that will be used to test the accuracy of your KNN implementation and its correct functioning (i.e. the testing data).

2. Store the remaining 80% of the records in the same way. In this case, use the names X_train andy_train for the arrays.
This is the data that your model will use as ground-truth knowledge (i.e. the training data, from which we extract the knowledge and that we will use for comparison).


In [7]:
import numpy as np

indices = np.random.choice(len(data), int(0.2 * len(data)), replace=False)
data_test = data[indices]
labels_test = labels[indices]

mask = np.ones(len(data), dtype=bool)
mask[indices] = False

data_train = data[mask]
labels_train = labels[mask]


3. Focus now on the KNN technique. 
From the next month, you will use the `scikit-learn` package. Many of its functionalities
are exposed via an object-oriented interface. With this paradigm in mind, implement now the KNN
algorithm and expose it as a Python class. The bare skeleton of your class should look like this (you
are free to add other methods if you want to).

```python
class KNearestNeighbors:
    def __init__(self, k):
        """
        Store the value of k in a attribute of the class and initialize other attributes.
        :param k : int, number of neighbors to consider.
        """
        pass # TODO: implement it!
    def fit(self, X, y):
        """
        Store the 'prior knowledge' of you model that will be used
        to predict new labels.
        :param X : input data points, ndarray, shape = (R,C).
        :param y : input labels, ndarray, shape = (R,).
        """
        pass # TODO: implement it!
    
    def predict(self, X):
        """Run the KNN classification on X.
        :param X: input data points, ndarray, shape = (N,C).
        :return: labels : ndarray, shape = (N,).
        """
        pass # TODO: implement it!

```


Implement the `__init__` and `fit` methods first. 
- In the `__init__` method, you should store the value of `k` in a private attribute of the class.
- In the `fit` method you should only store the training data in private attributes of the class.

4. Implement the `predict` method. The function receives as input a numpy array with N rows and C
columns, corresponding to N flowers. The method assigns to each row one of the three Iris species 
using the KNN algorithm, and returns the predicted species as a numpy array. 

    - For finding nearest neighbours, you can either re-use the previously defined `k_nearest_neighbors` function or 
implement a new one exploiting the numpy broadcasting capabilities in order to avoid iterating over the sample matrix `X`.
    - Then, assign the *predicted label* to each sample using a majority voting scheme, i.e., the label that appears most frequently among the k nearest neighbors. To do so you can use the `np.unique(neighbours_labels, return_count=True)` function that returns the unique labels and their counts. 
    - Finally, return the predicted labels as a numpy array.

In [8]:
class KNearestNeighbors:
    def __init__(self, k):
        """
        Store the value of k in a attribute of the class and initialize other attributes.
        :param k : int, number of neighbors to consider.
        """
        self.k = k
    def fit(self, X, y):
        """
        Store the 'prior knowledge' of you model that will be used
        to predict new labels.
        :param X : input data points, ndarray, shape = (R,C).
        :param y : input labels, ndarray, shape = (R,).
        """
        self.train_data = X
        self.train_label = y

    def __predict_this(self, el):
        nearest = k_nearest_neighbors(el, self.train_data, self.k)
        unique_labels, label_counts = np.unique(self.train_label[nearest], return_counts=True)
        prediction = unique_labels[label_counts.argsort()[-1]]
        return prediction

    def predict(self, X):
        """Run the KNN classification on X.
        :param X: input data points, ndarray, shape = (N,C).
        :return: labels : ndarray, shape = (N,).
        """\
        
        return np.array([
            self.__predict_this(el) for el in X
        ])


classifier = KNearestNeighbors(5)
classifier.fit(data_train, labels_train)
predictions = classifier.predict(data_test)

print(predictions)
print(labels_test)

[2 2 0 2 0 0 2 0 1 2 2 0 1 0 0 2 2 1 0 0 1 1 2 0 2 1 0 1 1 2]
[2 2 0 1 0 0 2 0 1 2 2 0 1 0 0 2 2 1 0 0 1 1 2 0 2 1 0 1 1 2]


5. Now let’s fit the KNN model with the X_train and y_train data. Then, try to use your KNN model
to predict the species for each record in X_test and store them in a nupy array called y_pred.
As we did in the previous lab, check how many Iris species in the array y_pred have been guessed correctly computing with respect to the ones in y_test computing the accuracy. 
    - A prediction is correct if `y_pred[i] == y_test[i]`. To get the accuracy then compute the ratio between the number of correct guesses and the total number of guesses is known. 
    - If all labels are assigned correctly ((y_pred == y_test).all() == True), the accuracy of the model is 100%. 
    - Instead, if none of the guessed species corresponds to the real one ((y_pred == y_test).any() == False), the accuracy is 0%


In [9]:
accuracy = (predictions == labels_test).sum() / labels_test.shape[0]
print("Accuracy:", accuracy)

Accuracy: 0.9666666666666667


6. ($\star$) As a software developer, you might want to increase the functionalities of your product and
publish newer versions over time. The better your code is structured and organized, the lower is the
effort to release updates.
As such,  extend your KNN implementation adding the parameter `distance`. This has to be one among:
    - Euclidean distance: $ euclidean(p,q) = \sqrt{\sum_{i=1}^{n} (p_i - q_i)^2} $
    - Manhattan distance: $ manhattan(p,q) = \sum_{i=1}^n |p_i - q_i|$
    - Cosine distance: $ cosine(p, q) = 1 - \frac{\sum_{i=1}^n p_i q_i}{ \sqrt{\sum^n_{i=1} p^2_i} \cdot \sqrt{\sum^n_{i=1} q_i^2}}$

If any of this distance is not already implemented in `numpy` implement it yourself

In [10]:
import numpy as np

def euclidean(p, q):
   return np.linalg.norm(p-q, axis=-1)

def manhattan(p, q):
   return np.linalg.norm(p-q, ord=1, axis=-1)
def cosine(p, q):
   return 1 - np.sum(p * q, axis=-1) / (np.linalg.norm(p, axis=-1) * np.linalg.norm(q, axis=-1))



7. ($\star$) Again, extend now your KNN implementation by adding the parameter `weights` to the constructor,
as shown below:

```python
class KNearestNeighbors:
    def __init__(self, k, distance_metric="euclidean", weights="uniform"):
        self.k = k
        self.distance_metric = distance_metric
        self.weights = weights
```

Change your KNN implementation to accept a new weighting scheme for the labels. If weights=
"distance", weight neighbor votes by the inverse of their distance (for the distance, again, use
distance_metric). The weight for a neighbor of the point p is:

$
w(p, n) = \frac{1}{distance\_metric(p, n)}
$

Instead, if the default is chosen (weights="uniform"), use the majority voting you already implemented
in Exercise 6.

<img src="https://mlarchive.com/wp-content/uploads/2022/09/img5.png">


In [11]:
class KNearestNeighbors:
    def __init__(self, k, distance_metric=euclidean, weights="uniform"):
        self.k = k
        self.distance_metric = distance_metric
        assert weights in ["uniform", "distance"]
        self.weights = weights

    def fit(self, X, y):
        """
        Store the 'prior knowledge' of you model that will be used
        to predict new labels.
        :param X : input data points, ndarray, shape = (R,C).
        :param y : input labels, ndarray, shape = (R,).
        """
        self.train_data = X
        self.train_label = y

    def __distance(self, p, q):
        distances = np.array([self.distance_metric(p, x) for x in q])
        return distances.argsort()[:self.k]
    
    def __predict_this(self, el):
        nearest = self.__distance(el, self.train_data)
        unique_labels, label_counts = np.unique(self.train_label[nearest], return_counts=True)
        if self.weights == "uniform":
            prediction = unique_labels[label_counts.argsort()[-1]]
        else:
            distances = np.array([self.distance_metric(el, x) for x in self.train_data[nearest]])
            epsilon = 1e-10
            weights = 1 / (distances + epsilon)
            label_weights = {label:0 for label in unique_labels}
            for label, weight in zip(self.train_label[nearest], weights):
                label_weights[label] += weight
            prediction = max(label_weights, key=label_weights.get)
        return prediction

    def predict(self, X):
        """Run the KNN classification on X.
        :param X: input data points, ndarray, shape = (N,C).
        :return: labels : ndarray, shape = (N,).
        """
        
        return np.array([
            self.__predict_this(el) for el in X
        ])

classifier = KNearestNeighbors(5, euclidean, "distance")
classifier.fit(data_train, labels_train)
predictions = classifier.predict(data_test)

accuracy = (predictions == labels_test).sum() / labels_test.shape[0]
print("Accuracy:", accuracy)

Accuracy: 0.9666666666666667


8. ($\star$) Test the modularity of the implementation applying it on a different dataset. Ideally, you should
not change the code of your KNN python class.
- Download the MNIST dataset and retain only 100 samples per digit. You will end up with a dataset of 1000 samples.
- Define again four numpy arrays as you did in Exercises 2 and 3.
- Apply your KNN as you did for the Iris dataset.
- Evaluate the accuracy on MNIST’s y_test.

In [12]:
# download MNIST dataset

# linux users
#! wget https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/mnist_test.csv -O mnist.csv

# windows users
! pip install wget
import os
import wget
MNIST_DATA_FILE = "mnist.csv"
if not os.path.exists(MNIST_DATA_FILE):
    wget.download("https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/mnist_test.csv", MNIST_DATA_FILE)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# extracting MNIST dataset
import csv
import numpy as np


data = []
labels = []
counters = [0]*10
with open(MNIST_DATA_FILE) as f:
    for line in csv.reader(f):
        label = int(line[0])
        if counters[label] >= 100: continue
        counters[label] += 1
        data.append(list(map(int, line[1:])))
        labels.append(label)

data = np.array(data)
labels = np.array(labels)




print(data.shape, labels.shape)

(1000, 784) (1000,)


In [14]:
# define four numpy arrays x_train, y_train, x_test, y_test
indices = np.random.choice(len(data), int(0.2 * len(data)), replace=False)
data_test = data[indices]
labels_test = labels[indices]

mask = np.ones(len(data), dtype=bool)
mask[indices] = False

data_train = data[mask]
labels_train = labels[mask]

In [15]:
# Apply KNN on MNIST
classifier = KNearestNeighbors(5, cosine, "distance")
classifier.fit(data_train, labels_train)
predictions = classifier.predict(data_test)

correct = 0
for i, p in enumerate(predictions):
    if p == labels_test[i]: correct+=1

print(f"Accuracy: {correct / len(predictions)}")

Accuracy: 0.9
